In [1]:
from mongoengine import *
from dotenv import load_dotenv
import os
import requests
import pandas as pd
connect("reefscape_predictor") 

load_dotenv()

tba_api_key = os.getenv("TBA_API_KEY")


In [15]:
class Team(Document):
    event = StringField(required=True)
    team = IntField(required=True)
    epa = FloatField(required=True)
    total_points = FloatField(required=True)
    auto_points = FloatField(required=True)
    teleop_points = FloatField(required=True)
    endgame_points = FloatField(required=True)
    rank = IntField(required=True)
    winrate = FloatField(required=True)
    coral_count = FloatField()
    l4_count = FloatField()
    l3_count = FloatField()
    algae_count = FloatField()
    opr = FloatField()
    ccwm = FloatField()
    
    def __str__(self):
        return f'''Team {self.team}
                    EPA: {self.epa}
                    Total Points: {self.total_points}
                    Auto Points: {self.auto_points}
                    Teleop Points: {self.teleop_points}
                    Endgame Points: {self.endgame_points}
                    Global Rank: {self.rank}
                    Winrate: {self.winrate}
                    OPR: {self.opr}
                    CCWM: {self.ccwm}
                    '''

In [16]:

class Event(Document):
    name = StringField(required=True)
    key = StringField(required=True)
    week = IntField(required=True)
    oprs = DictField(required=True)
    coprs = DictField(required=True)
    
    def __str__(self):
        return f'''
                {self.name} week {self.week}... {self.key}
                '''
    

In [17]:
class Match(Document):
    key = StringField(required=True)
    red_alliance = ListField(ReferenceField(Team), required=True, min_length=3, max_length=3)
    blue_alliance = ListField(ReferenceField(Team), required=True, min_length=3, max_length=3)
    blue_score = FloatField(required=True)
    red_score = FloatField(required=True)

    def __str__(self):
        return f'''Match:
                    Red Alliance: {[team.team for team in self.red_alliance]} scored {self.red_score}
                    Blue Alliance: {[team.team for team in self.blue_alliance]} scored {self.blue_score}
                    Winner was {"red" if self.red_score > self.blue_score else "blue"}                    
                '''

In [18]:
def get_all_season_events():
    try:
        
        req = requests.get(
            "https://www.thebluealliance.com/api/v3/events/2025",
            {"X-TBA-Auth-Key": tba_api_key},
        )
        
        data = req.json()
        
        return data
    except requests.ConnectionError as e:
        print(e)
        
events = get_all_season_events()


In [19]:
def get_oprs(event_key: str):
    try:
        req = requests.get(f"https://www.thebluealliance.com/api/v3/event/{event_key}/oprs",  {"X-TBA-Auth-Key": tba_api_key})
        
        oprs = req.json()
        
        req = requests.get(f"https://www.thebluealliance.com/api/v3/event/{event_key}/coprs",  {"X-TBA-Auth-Key": tba_api_key})
        
        coprs = req.json()
        
        return oprs, coprs
    except requests.ConnectionError as e:
        print(e)
        

In [91]:
def save_event(event_key):
    if len(Event.objects(key=event_key)) > 0:
        return Event.objects(key=event_key)[0]
    
    oprs, coprs = get_oprs(event_key)
    
    res = requests.get("https://www.thebluealliance.com/api/v3/event/2025mxto", {"X-TBA-Auth-Key": tba_api_key})
    
    response = res.json()
    print(response["week"])
    event_obj = Event(
                    week=response["week"],
                    name=response["name"],
                    key=response["key"],
                    oprs=oprs,
                    coprs=coprs,
                )
    # print(event_obj)
    event_obj.save()
    

In [ ]:
count = 0
for event in events:
    week: int = event["week"]
    # print(type(week))
    # print(week)

    if week is not None:
        if week < 3:
            oprs, coprs = get_oprs(event["key"])
            if len(oprs) > 0 and len(coprs) > 0:
                # print(event["key"], week, "\n")
                event_obj = Event(
                    week=event["week"],
                    name=event["name"],
                    key=event["key"],
                    oprs=oprs,
                    coprs=coprs,
                )
                event_obj.save()
                count += 1
        else:
            print(event["key"], week)

    else:
        print(event["key"], week)

print("COUNT", count)
# if week < 4:
#     event_obj = Event(week = event["week"], name=event["name"], key=event["key"])
#     event_obj.save()

In [21]:
print(len(Event.objects()))


95


In [22]:
def get_events_match_keys(event_key:str):
    try:
        req = requests.get(f"https://www.thebluealliance.com/api/v3/event/{event_key}/matches/keys", 
                           {"X-TBA-Auth-Key": tba_api_key})
        # print(req.json())
        return req.json()
        
    except requests.ConnectionError as e:
        print(f"Error fetching {event_key} keys: \n{e}")

In [23]:
# print(Event.objects(key="2025mxle")[0].oprs["oprs"]["frc2283"])
print(Event.objects(key="2025mxle")[0].coprs["Total Algae Count"]["frc4731"])

6.793968856165202


In [96]:


def get_team_info(team_number: int, event_key: str):
    mongo_team: list = Team.objects(team=team_number, event=event_key)
    if len(mongo_team) > 0:
        return Team.objects(team=team_number)[0]
    try:
        req = requests.get(f"https://api.statbotics.io/v3/team_year/{team_number}/2025")
        
        team_dict = req.json()
        if len(Event.objects(key=event_key)) < 1:
            save_event(event_key)
        ccwm = Event.objects(key=event_key)[0].oprs["ccwms"][f"frc{team_number}"]
        opr = Event.objects(key=event_key)[0].oprs["oprs"][f"frc{team_number}"]
        l3_count = Event.objects(key=event_key)[0].coprs["L3 Coral Count"][f"frc{team_number}"]
        l4_count = Event.objects(key=event_key)[0].coprs["L4 Coral Count"][f"frc{team_number}"]
        coral_count = Event.objects(key=event_key)[0].coprs["Total Coral Count"][f"frc{team_number}"]
        algae_count = Event.objects(key=event_key)[0].coprs["Total Algae Count"][f"frc{team_number}"]
        team_info = {
            "event": event_key,
            "team": team_dict["team"],
            "epa": team_dict["epa"]["norm"],
            "total_points": team_dict["epa"]["breakdown"]["total_points"],
            "auto_points": team_dict["epa"]["breakdown"]["auto_points"],
            "teleop_points": team_dict["epa"]["breakdown"]["teleop_points"],
            "endgame_points": team_dict["epa"]["breakdown"]["endgame_points"],
            "rank": team_dict["epa"]["ranks"]["total"]["rank"],
            "winrate": team_dict["record"]["winrate"],
            "ccwm": ccwm,
            "opr": opr,
            "l3_count": l3_count,
            "l4_count": l4_count,
            "coral_count": coral_count,
            "algae_count": algae_count,
            }
        team_obj = Team.from_json(str(team_info).replace("'", '"'))
        team_obj.save() 
        
        return team_obj
    except requests.ConnectionError as e:
        print(f"Error fetching team: {e}")
        return "error"
    except KeyError as k:
        print(f"Skipping {team_number}'s matches at {event_key} ")
        return "error"

In [280]:
print(get_team_info(9992, "2025ncash"))

Skipping 9992's matches at 2025ncash 
error


In [105]:
def get_match_info(match_key: str):
    mongo_team: list = Match.objects(key=match_key)
    if len(mongo_team) > 0:
        return Match.objects(key=match_key)[0]
    try:
        response = requests.get(
            f"https://www.thebluealliance.com/api/v3/match/{match_key}",
            {
                "X-TBA-Auth-Key": tba_api_key
            }
        )
        info = response.json()
        match = Match()
        match.key = match_key
        red_info = info["alliances"]["red"]
        blue_info = info["alliances"]["blue"]
        match.red_alliance = [get_team_info(int(team[3:]), match_key[:match_key.index("_")]) for team in red_info["team_keys"]]
        match.blue_alliance = [get_team_info(int(team[3:]), match_key[:match_key.index("_")]) for team in blue_info["team_keys"]]
        
        if "error" in str(match.red_alliance) or "error" in str(match.blue_alliance):
            print(f"Skipping match {match_key} due to team info error.")
            return "error"
        
        match.red_score = red_info["score"]
        match.blue_score = blue_info["score"]
        # match.save()
        return match
    except requests.ConnectionError as e:
        print(f"Error fetching match: {match_key}")
        
        return "error"



In [86]:
print(len(Event.objects(key="2025mxto")) < 1)

# botbusters = get_team_info(4403, "2025mxto")
# print(botbusters)
# info = get_match_info("2025mxto_qm8")
# print(info)

True


In [272]:
def save_event_matches(event_key: str):
    matches = get_events_match_keys(event_key)
    for match in matches:
        # print(match)
        get_match_info(match)
    

In [28]:
print(len(Match.objects()))

8063


In [ ]:
count = 0
for event in Event.objects():
    print(event)
    save_event_matches(event_key=event.key)
    count += 1
    print(count)

In [29]:

class CompleteMatch(Document):
    key = StringField(required=True)
    red3_team = IntField(required=True)
    red2_team = IntField(required=True)
    red1_team = IntField(required=True)
    blue3_team = IntField(required=True)
    blue2_team = IntField(required=True)
    blue1_team = IntField(required=True)
    red3_epa = FloatField(required=True)
    red2_epa = FloatField(required=True)
    red1_epa = FloatField(required=True)
    blue3_epa = FloatField(required=True)
    blue2_epa = FloatField(required=True)
    blue1_epa = FloatField(required=True)
    red3_total_points = FloatField(required=True)
    red2_total_points = FloatField(required=True)
    red1_total_points = FloatField(required=True)
    blue3_total_points = FloatField(required=True)
    blue2_total_points = FloatField(required=True)
    blue1_total_points = FloatField(required=True)
    red3_auto_points = FloatField(required=True)
    red2_auto_points = FloatField(required=True)
    red1_auto_points = FloatField(required=True)
    blue3_auto_points = FloatField(required=True)
    blue2_auto_points = FloatField(required=True)
    blue1_auto_points = FloatField(required=True)
    red3_teleop_points = FloatField(required=True)
    red2_teleop_points = FloatField(required=True)
    red1_teleop_points = FloatField(required=True)
    blue3_teleop_points = FloatField(required=True)
    blue2_teleop_points = FloatField(required=True)
    blue1_teleop_points = FloatField(required=True)
    red3_endgame_points = FloatField(required=True)
    red2_endgame_points = FloatField(required=True)
    red1_endgame_points = FloatField(required=True)
    blue3_endgame_points = FloatField(required=True)
    blue2_endgame_points = FloatField(required=True)
    blue1_endgame_points = FloatField(required=True)
    red3_rank = IntField(required=True)
    red2_rank = IntField(required=True)
    red1_rank = IntField(required=True)
    blue3_rank = IntField(required=True)
    blue2_rank = IntField(required=True)
    blue1_rank = IntField(required=True)
    red3_winrate = FloatField(required=True)
    red2_winrate = FloatField(required=True)
    red1_winrate = FloatField(required=True)
    blue3_winrate = FloatField(required=True)
    blue2_winrate = FloatField(required=True)
    blue1_winrate = FloatField(required=True)
    red3_coral_count = FloatField()
    red2_coral_count = FloatField()
    red1_coral_count = FloatField()
    blue3_coral_count = FloatField()
    blue2_coral_count = FloatField()
    blue1_coral_count = FloatField()
    red3_l4_count = FloatField()
    red2_l4_count = FloatField()
    red1_l4_count = FloatField()
    blue3_l4_count = FloatField()
    blue2_l4_count = FloatField()
    blue1_l4_count = FloatField()
    red3_l3_count = FloatField()
    red2_l3_count = FloatField()
    red1_l3_count = FloatField()
    blue3_l3_count = FloatField()
    blue2_l3_count = FloatField()
    blue1_l3_count = FloatField()
    red3_algae_count = FloatField()
    red2_algae_count = FloatField()
    red1_algae_count = FloatField()
    blue3_algae_count = FloatField()
    blue2_algae_count = FloatField()
    blue1_algae_count = FloatField()
    red3_opr = FloatField()
    red2_opr = FloatField()
    red1_opr = FloatField()
    blue3_opr = FloatField()
    blue2_opr = FloatField()
    blue1_opr = FloatField()
    red3_ccwm = FloatField()
    red2_ccwm = FloatField()
    red1_ccwm = FloatField()
    blue3_ccwm = FloatField()
    blue2_ccwm = FloatField()
    blue1_ccwm = FloatField()
    red_score = FloatField()
    blue_score = FloatField()
    def __str__(self):
        return (
            f"Match {self.key}: "
            f"Red {self.red1_team}, {self.red2_team}, {self.red3_team} ({self.red_score}) - "
            f"Blue {self.blue1_team}, {self.blue2_team}, {self.blue3_team} ({self.blue_score})"
        )

In [102]:

def convert_match_to_complete(match_obj: Match):
    """Convierte un match de la base de datos en una instancia de CompleteMatch con datos embebidos."""
    
    match_data = {
        "key": match_obj.key,
        "blue_score": match_obj.blue_score,
        "red_score": match_obj.red_score,
    }

    for color in ["red", "blue"]:
        for pos in range(1, 4): 
            team_ref: Team = match_obj.blue_alliance[pos-1]
            team_data: Team = Team.objects(team=team_ref.team)[0]

            match_data[f"{color}{pos}_team"] = team_ref.team
            match_data[f"{color}{pos}_epa"] = team_data.epa
            match_data[f"{color}{pos}_total_points"] = team_data.total_points
            match_data[f"{color}{pos}_auto_points"] = team_data.auto_points
            match_data[f"{color}{pos}_teleop_points"] = team_data.teleop_points
            match_data[f"{color}{pos}_endgame_points"] = team_data.endgame_points
            match_data[f"{color}{pos}_rank"] = team_data.rank
            match_data[f"{color}{pos}_winrate"] = team_data.winrate
            match_data[f"{color}{pos}_coral_count"] = team_data.coral_count
            match_data[f"{color}{pos}_l4_count"] = team_data.l4_count
            match_data[f"{color}{pos}_l3_count"] = team_data.l3_count
            match_data[f"{color}{pos}_algae_count"] = team_data.algae_count
            match_data[f"{color}{pos}_opr"] = team_data.opr
            match_data[f"{color}{pos}_ccwm"] = team_data.ccwm
            
    # print(match_data)
    final_match = CompleteMatch(**match_data)
    final_match.save()
    return final_match

In [ ]:
count = 0
for match in Match.objects():
    convert_match_to_complete(match)
    count += 1
    print(count, match.key)


In [298]:
print(len(CompleteMatch.objects()))

8063


In [119]:
def get_match_series(match_key: str) -> pd.Series:
   
    # Primero, obtener el CompleteMatch para el match_key dado.
    complete_matches = CompleteMatch.objects(key=match_key)
    if len(complete_matches) > 0:
        complete_match = complete_matches[0]
    else:
        match_obj = get_match_info(match_key)
        print(match_obj)
        if match_obj == "error":
            raise ValueError(f"Error al obtener el match con key {match_key}")
        complete_match = convert_match_to_complete(match_obj)
        complete_match = CompleteMatch.objects(key=match_key)[0]
    
    # Extraer los datos del objeto (se asume que se guardaron en _data)
    match_dict = complete_match._data.copy()
    if '_id' in match_dict:
        del match_dict['_id']
    
    # Definir el orden deseado de columnas:
    columns_order = [
        "red3_epa", "red2_epa", "red1_epa",
        "blue3_epa", "blue2_epa", "blue1_epa",
        "red3_total_points", "red2_total_points", "red1_total_points",
        "blue3_total_points", "blue2_total_points", "blue1_total_points",
        "red3_auto_points", "red2_auto_points", "red1_auto_points",
        "blue3_auto_points", "blue2_auto_points", "blue1_auto_points",
        "red3_teleop_points", "red2_teleop_points", "red1_teleop_points",
        "blue3_teleop_points", "blue2_teleop_points", "blue1_teleop_points",
        "red3_endgame_points", "red2_endgame_points", "red1_endgame_points",
        "blue3_endgame_points", "blue2_endgame_points", "blue1_endgame_points",
        "red3_rank", "red2_rank", "red1_rank",
        "blue3_rank", "blue2_rank", "blue1_rank",
        "red3_winrate", "red2_winrate", "red1_winrate",
        "blue3_winrate", "blue2_winrate", "blue1_winrate",
        "red3_coral_count", "red2_coral_count", "red1_coral_count",
        "blue3_coral_count", "blue2_coral_count", "blue1_coral_count",
        "red3_l4_count", "red2_l4_count", "red1_l4_count",
        "blue3_l4_count", "blue2_l4_count", "blue1_l4_count",
        "red3_l3_count", "red2_l3_count", "red1_l3_count",
        "blue3_l3_count", "blue2_l3_count", "blue1_l3_count",
        "red3_algae_count", "red2_algae_count", "red1_algae_count",
        "blue3_algae_count", "blue2_algae_count", "blue1_algae_count",
        "red3_opr", "red2_opr", "red1_opr",
        "blue3_opr", "blue2_opr", "blue1_opr",
        "red3_ccwm", "red2_ccwm", "red1_ccwm",
        "blue3_ccwm", "blue2_ccwm", "blue1_ccwm",
        "red_score", "blue_score"
    ]
    
    # Crear un diccionario ordenado según la lista deseada.
    ordered_dict = {col: match_dict[col] for col in columns_order if col in match_dict}
    
    # Convertir a pandas Series.
    match_series = pd.Series(ordered_dict)
    return match_series


In [75]:
h = get_match_series("2025mxle_qm1")
print(h)

red3_epa      1469.000000
red2_epa      1514.000000
red1_epa      1724.000000
blue3_epa     1469.000000
blue2_epa     1514.000000
                 ...     
blue3_ccwm     -10.713549
blue2_ccwm     -10.150860
blue1_ccwm      20.894840
red_score       68.000000
blue_score      99.000000
Length: 80, dtype: float64


In [60]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score


device = 'mps' if torch.mps.is_available() else 'cpu'
print(device)

mps


In [61]:
class MyNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2) 
        )

    def forward(self, x):
        return self.model(x)


In [63]:
model = MyNet(78).to(device)
model.load_state_dict(torch.load("second_iteration/model.pt"))

<All keys matched successfully>

In [120]:
def inferencia(fila, model, scaler, threshold=5.0):
   

    features = fila.drop(labels=["blue_score", "red_score"]).values.reshape(1, -1)

    features_scaled = scaler.transform(features)

    tensor_features = torch.tensor(features_scaled, dtype=torch.float32).to(device)

    model.eval()
    with torch.no_grad():
        prediction = model(tensor_features)

    predicted_scores = prediction.cpu().numpy().flatten()

    true_scores = fila[["blue_score", "red_score"]].values.astype(np.float32)

    error_abs = np.abs(true_scores - predicted_scores)

    accuracy = 100.0 if (error_abs < threshold).all() else 0.0

    actual_winner = "blue" if true_scores[0] > true_scores[1] else "red"
    
    pred_winner = "blue" if predicted_scores[0] > predicted_scores[1] else "red"
    print(
        f"Predicción -> blue_score: {predicted_scores[0]:.2f}, red_score: {predicted_scores[1]:.2f}"
    )
    print(
        f"Valor real -> blue_score: {true_scores[0]:.2f}, red_score: {true_scores[1]:.2f}"
    )
    
    print("Win prediction was", actual_winner == pred_winner)
    return actual_winner == pred_winner

In [77]:
data_df = pd.read_csv("second_iteration/clean_data.csv")
data_df.dropna(inplace=True)
original_df = data_df.copy()

x = data_df.drop(["blue_score", "red_score"], axis=1).values
y = data_df[["blue_score", "red_score"]].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15,)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)


In [121]:
matchh = get_match_series("2025mxto_qm31")

# print(matchh)
inferencia(matchh, model, scaler)

Predicción -> blue_score: 22.53, red_score: 43.85
Valor real -> blue_score: -1.00, red_score: -1.00
Win prediction was True


True

In [112]:
for match in CompleteMatch.objects():
    if "2025mxto" in str(match.key):
        match.delete()